# **Python Azure 01**
---

💡 Blob Storage에 파일을 올리고, 확인하고 Local Storage로 다운로드 받는 과정을 실습해본다.

- 필요한 패키지를 설치한다.

In [28]:
# 필요한 패키지 설치
!pip install azure-storage-blob azure-identity

In [29]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

- Azure에서 연결 문자열을 가져와 연결 문자열을 이용하여 Azure Blob 서버에 접속한다.

In [30]:
# Azure에서 복사한 '연결 문자열'을 이용하여 Blob 서버에 접속하기
connect_str = '***************************************************'             # 연결 문자열 ✅
blob_service_client = BlobServiceClient.from_connection_string(connect_str)     # 연결 문자열을 이용해서 Blob Service에 연결

- 원격으로 Blob Storage에 컨테이너를 생성한다.

In [31]:
container_name = 'test33container'

container_client = blob_service_client.create_container(container_name)

- Local Storage에서 파일을 생성한다.

In [32]:
# Create a local directory to hold blob data
local_path = './data'
os.mkdir(local_path)

# Create a file in the local directory
local_file_name = 'welcome' + '.txt'
upload_file_path = os.path.join(local_path, local_file_name)

# Write text to the file
file = open(upload_file_path, mode='w')
file.write('Welcome to Python')
file.close()     # Close를 안하면 파일이 나중에 잘 깨진다. (0B 파일들)

- Blob Storage에 Local Storage에서 만든 파일을 업로드한다.

In [33]:
blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)
with open(file=upload_file_path, mode='rb') as data:    # rb : read binary
    blob_client.upload_blob(data)

- Blob Storage에 있는 파일들의 목록을 확인한다.

In [34]:
# blob 파일의 리스트를 확인한다. 
print('\nListing blobs')

blob_list = container_client.list_blobs()
for blob in blob_list:
  print('\t' + blob.name)


Listing blobs
	welcome.txt


- Blob Storage에 있는 파일을 Local Storage로 다운로드 받는다. 그리고 다운로드 받은 파일의 경로를 출력한다.
    - 이때, 파일의 이름을 `DOWNLOAD.txt` 로 변경한다.

In [35]:
# Download the blob to a local file
# Add 'DOWNLOAD' befere the .txt extendsion so you can see both files in the data directory 

download_file_path = os.path.join(local_path, str.replace(local_file_name,'.txt','_DOWNLOAD.txt'))
print(download_file_path)
container_client = blob_service_client.get_container_client(container=container_name)

with open(file=download_file_path, mode='wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall())

./data\welcome_DOWNLOAD.txt


- 지금까지 생성하고 생성된 파일들과 디렉터리를 지운다.

In [36]:
# Clean Up
print('\nType \'yes\' to begin clean up')
if input() == 'yes':

  print('Deleting blob container...')
  container_client.delete_container()     # container_client를 지운다.

  print('Deleteing the local source and downloaded files....')
  os.remove(upload_file_path)          # 업로드한 파일 지우기
  os.remove(download_file_path)        # 다운로드한 파일 지우기
  os.rmdir(local_path)                 # 디렉터리 지우기

  print('Done')


Type 'yes' to begin clean up
Deleting blob container...
Deleteing the local source and downloaded files....
Done
